# Sequential and Parallel Composition

## Introduction

In this notebook, we will: 

- Define sequential composition and parallel composition.
- Illustrate sequential composition and parallel composition in practice through polars queries.
- Compare and contrast sequential and parallel composition.

Further reading:

- [Programming Differential Privacy: Properties of Differential Privacy](https://programming-dp.com/ch4.html)
- [Concurrent Composition of Differential Privacy](https://eprint.iacr.org/2021/1196.pdf)

### Composition


Before we get into the different types of composition, let's understand why composition is important and what it means. We will rarely only want one differentially private statistic; instead, we will compute multiple statistics on the same dataset. 

If we run many different differentially private algorithms on the same dataset, the resulting **composed** algorithm is also differentially private. So composition is a useful tool because it enables us to analyze the privacy properties of algorithms designed in this way. 

### Key Terms

**Noninteractive**: All queries are composed and executed at once. Specifically, take $F$ as a noninteractive algorithm composed of individual queries $F = \text{Comp}(F_1, F_2, \ldots, F_{k-1})$ and dataset x as the input dataset. $F(x)$ is our output, an algorithm that executes all the queries. 

**Interactive**: In an interactive algorithm, the queries are not defined in advance, and they can be determined based on the results from previous queries. This is an **adaptive** scenario. 

Interactivity can be viewed as a superset that includes another kind of variation: non-adaptive, adaptive, or concurrent. 

- A **non-adaptive** scenario is where you simply want to compute statistics such as the mean, median, and sum of particular columns in a dataset. 

- An **adaptive** scenario allows you to choose the next query after releasing the previous query. For example, analysis begins with computing particular statistics about a column, such as "income," and then later queries depend on the results from the prior queries. For example, if you notice that the mean and median are extremely different, you may explore the distribution of the data. Otherwise, you may choose to focus your queries on other, more interesting aspects of your data. However, adaptive queries cannot be run concurrently and have a sequential constraint. 

- **Concurrent** composition addresses this limitation. There is ongoing research about this mechanism. 

Interactive differential privacy is considered the "right" modeling for many applications since it is required for implementing many DP mechanisms such as the Sparse Vector Technique. 

Although the non-adaptive, adaptive, and concurrent axis is a significant axis of variation, this section will focus primarily on sequential and parallel composition. 

## Set Up

In [3]:
import opendp.prelude as dp
import numpy as np
import polars as pl 

dp.enable_features("contrib")

In [4]:
# Fetch and download data. 
![ -e sample_FR_LFS.csv ] || ( curl 'https://github.com/opendp/dp-test-datasets/blob/master/data/sample_FR_LFS.csv.zip?raw=true' --location --output sample_FR_LFS.csv.zip; unzip sample_FR_LFS.csv.zip )

df = pl.scan_csv("sample_FR_LFS.csv", ignore_errors=True)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 5933k  100 5933k    0     0   913k      0  0:00:06  0:00:06 --:--:--  943k
Archive:  data.zip
  inflating: sample_FR_LFS.csv       
  inflating: __MACOSX/._sample_FR_LFS.csv  


In [5]:
context = dp.Context.compositor(
    data=df,
    privacy_unit=dp.unit_of(contributions=36),
    privacy_loss=dp.loss_of(epsilon=1.0),
    split_evenly_over=10,
    margins={
        ("ILOSTAT", ): dp.Margin(public_info="lengths", max_partition_length=60_000_000, max_num_partitions=3),
        ("YEAR", "QUARTER", "ILOSTAT",): dp.Margin(public_info="lengths", max_partition_length=60_000_000, max_partition_contributions=1, max_num_partitions=1),
        (): dp.Margin(public_info="lengths", max_partition_length=60_000_000, max_num_partitions=1),
    },
)

# Filter out rows with 99 as a null value. 
df = df.filter(pl.col("HWUSUAL")!=99)

## Sequential Composition

Sequential composition is a major property of differential privacy because it bounds the total privacy costs of computing multiple differentially private queries on the same data. It's important because it allows the design of algorithms that refer to the data multiple times. 

It's an extremely useful way to get an **upper** bound on the total ε of many queries, but keep in mind that the actual impact on privacy may be lower. 

Assume that function 1, $F1$ satisfies ε-dp and function 2, $F2$, also satisfies ε-dp. 

Then a mechanism where we compose functions, $g(x)$ = ($F1$(x), $F2$(x)), we get 2 * ε-dp.

Sequential composition can be noninteractive and interactive. 

### Noninteractive Sequential Composition

We can execute multiple queries at once using the `select` or `agg` methods. To start off, let's calculate the actual and differentially private mean and median for the variable `HWUSUAL`, which is the number of hours per week usually worked in the main job by respondents that were working for pay at the time of the survey.

Notes: 
- Reference the [EU Labour Force Survey User Guide](https://ec.europa.eu/eurostat/documents/1978984/6037342/EULFS-Database-UserGuide.pdf) for more information about variables. 
- Do not calculate the actual mean to impute null values. We shouldn't reference our data after creating the compositor. 

In [6]:
candidates = list(range(25,56))

actual_hours = df.select(
    pl.col("HWUSUAL").fill_null(40.0).mean().alias("Actual Mean Hours Worked"), 
    pl.col("HWUSUAL").fill_null(40.0).median().alias("Actual Median Hours Worked"),
).collect()

actual_hours

Actual Mean Hours Worked,Actual Median Hours Worked
f64,f64
37.634091,37.0


In [7]:
dp_hours = context.query().select(
    pl.col("HWUSUAL").fill_null(40.0).dp.mean((1,45)).alias("DP Mean Hours Worked"), 
    pl.col("HWUSUAL").fill_null(40.0).dp.median(candidates).alias("DP Median Hours Worked"),
).release().collect()

dp_hours

DP Mean Hours Worked,DP Median Hours Worked
f64,i64
41.350777,54


## Interactive Sequential Composition

In cases where you want to apply interactive sequential composition, you can use the `dp.c.make_sequential_composition` method. There are two ways to implement sequential composition: using transformations and measurements directly and passing in polars queries. 

Also note, that the following queries are interactive, but they should not be assumed to be adaptive. 

### Interactive Sequential Composition with Transformations and Measurements

The interactive sequential composition doesn't use the compositor we defined at the start of this notebook. Instead, we construct a composition through transformations and measurements, combinator elements present in lower-level frameworks in the OpenDP library. 

We will also define a `queryable` in the following code. It can essentially be thought of as a class that executes the queries and tracks the privacy expenditure. 

In [8]:
# Define the compositor by specifying parameters. 
sc_meas = dp.c.make_sequential_composition(
    input_domain=dp.vector_domain(dp.atom_domain(T=float)),
    input_metric=dp.symmetric_distance(),
    output_measure=dp.max_divergence(T=float),
    # D_in is the upper bound on distances. 
    d_in=1,
    # d_mids is the privacy consumption for each query. 
    d_mids=[1., 1., 2.]
)

print("Privacy Consumption of Entire Composition: " + str(sc_meas.map(1)))

# 40 is chosen since it represents the average working hours. 
# A lower value such as 0 may skew the results
hours_list = list(np.array(df.select(
                                    pl.col("HWUSUAL").
                                    fill_null(40.0).
                                    fill_nan(40.0)).
                                    collect()
                                # The reshape of the list flattens it. 
                                # Which is required for the queryable input. 
                                ).reshape(-1))

# This is a queryable that takes in the data similar to the context. 
sc_qbl = sc_meas(hours_list)

Privacy Consumption of Entire Composition: 4.0


In [9]:
# Construct a measurement for mean. 
input_space = dp.vector_domain(dp.atom_domain(T=float)), dp.symmetric_distance()
count_meas = input_space >> dp.t.then_count() >> dp.m.then_laplace(scale=1.0)
count_release = sc_qbl(count_meas)
print("DP count: ",count_release)

mean_meas = (
    input_space >> 
    dp.t.then_clamp((0.,79.)) >> 
    dp.t.then_resize(size=count_release, constant=40.) >> 
    dp.t.then_mean() >> 
    dp.m.then_laplace(1.))
print("DP Mean:", sc_qbl(mean_meas))

DP count:  78706
DP Mean: 37.83963576379795


In [10]:
# Construct a measurement for median. 
discrete_scores = dp.t.make_quantile_score_candidates(dp.vector_domain(dp.atom_domain(T=float)), 
                                    dp.symmetric_distance(), 
                                    np.array(candidates).astype(float), 
                                    0.5)
input_space = dp.vector_domain(dp.atom_domain(T=dp.usize), 31), dp.linf_distance(T=dp.usize)
select_index_measurement = dp.m.make_report_noisy_max_gumbel(*input_space, scale=1.0, optimize='min')

median_meas = discrete_scores >> select_index_measurement >> (lambda index: candidates[index])
print("DP Median:", sc_qbl(median_meas))

DP Median: 37


That was a lot of code! But as you saw, we could change our queries depending on the outputs of other queries, illustrating interactive sequential composition.

### Interactive Sequential Composition with Polars Queries

You can also implement interactive sequential composition with polars queries! One of the first steps is to specify the domain of our data frame and add a margin specifying the parameters for the data when it's grouped by `ILOSTAT`. This is **in addition** to specifying it in the compositor. 

In [11]:
# Get the domain of the datafarame and specify parameters. 
df_domain = dp.domain_of(df, infer=True)

df_domain = dp.with_margin(df_domain, by=[], 
                           public_info="lengths", 
                           max_partition_length=1, 
                           max_num_partitions=1)

proposed_plan = context.query().select(
    pl.col("HWUSUAL").fill_null(40.0).dp.mean((1,98)).alias("DP Mean Hours Worked"), 
    pl.col("HWUSUAL").fill_null(40.0).dp.median(candidates).alias("DP Median Hours Worked"),
)


Next, we set up a measurement to execute the sequential queries and pass them in our `df_domain` and `proposed_plan`. We can pass our data to this measurement and then collect our results. 

In [12]:
#TODO: Fix Parameters

# Polars Lazyframe
m_polars = dp.m.make_private_lazyframe(
    input_domain=df_domain, 
    input_metric=dp.symmetric_distance(), 
    output_measure=dp.max_divergence(T=float), 
    lazyframe=proposed_plan, 
    global_scale=45.
)

# Measurement for sequential composition. 
m_comp = dp.c.make_sequential_composition(
    m_polars.input_domain,
    m_polars.input_metric,
    m_polars.output_measure,
    2, 
    [3., 3.]
)

qbl_comp = m_comp(df)
qbl_comp(m_polars).collect()

DP Mean Hours Worked,DP Median Hours Worked
f64,i64
37.636212,50


## Parallel Composition

Parallel composition can be viewed as an alternative to sequential composition since it's another way to calculate a bound on the total privacy cost of multiple queries. 

Parallel composition involves partitioning the input dataset into disjoint partitions and applying a differentially private mechanism on each chunk independent of all other chunks. Assuming each individual is represented once in the dataset, their data will appear in exactly one chunk. If there are $k$ chunks, there will be $k$ runs of the differentially private mechanisms.

To apply parallel composition, we need to set up our margin or define our query to allow each individual to influence at most 1 partition, or appear at most in 1 chunk. This is different from earlier in the notebook where we assumed 36 contributions per user since each user was represented once per year-quarter across 13 years. 


### Noninteractive Parallel Composition

In the following example, each individual is represented just one each year, quarter, and labor status. Moreover, this example also satisfies the properties of noninteractive sequential composition since all the queries are executed simultaneously. 

In [13]:
context.query().group_by(["YEAR","QUARTER","ILOSTAT"]).agg(
    pl.col("HWUSUAL").fill_null(40.).dp.mean((1,98)).alias("DP Mean Hours Worked"),
    pl.col("HWUSUAL").fill_null(40.).dp.median(candidates).alias("DP Median Hours Worked")
).release().collect().sort(["YEAR", "QUARTER"])

YEAR,QUARTER,ILOSTAT,DP Mean Hours Worked,DP Median Hours Worked
i64,i64,i64,f64,i64
2004,1,3,98.393258,50
2004,1,1,37.256882,40
2004,1,2,94.884377,31
2004,1,9,98.780115,26
2004,2,9,93.57499,42
…,…,…,…,…
2013,3,9,96.685363,45
2013,4,3,98.367484,29
2013,4,1,38.106898,34


### Interactive Parallel Composition

We'll illustrate interactive parallel composition using the polar-interactivity methods. Much of the code is similar to interactive sequential composition. The primary change is the domain and proposed plan. 

In [14]:
#get the domain of the datafarame
lf_domain = dp.domain_of(df, infer=True)
lf_domain = dp.with_margin(lf_domain, by=["YEAR","QUARTER","ILOSTAT"], 
                           public_info="lengths", 
                           max_partition_length=50, 
                           max_num_partitions=3)

proposed_plan = context.query().group_by(["YEAR","QUARTER","ILOSTAT"]).agg(
    pl.col("HWUSUAL").fill_null(40.).dp.mean((1,98), 8).alias("DP Mean Hours Worked"),
    pl.col("HWUSUAL").fill_null(40.).dp.median(candidates, 8).alias("DP Median Hours Worked")
)
# IN SERVER
m_polars = dp.m.make_private_lazyframe(
    input_domain=lf_domain, 
    input_metric=dp.symmetric_distance(), 
    output_measure=dp.max_divergence(T=float), 
    lazyframe=proposed_plan, 
    global_scale=1.
)

m_comp = dp.c.make_sequential_composition(
    m_polars.input_domain,
    m_polars.input_metric,
    m_polars.output_measure,
    2, 
    [18.0, 18.0]
)


qbl_comp = m_comp(df)
qbl_comp(m_polars).collect()

YEAR,QUARTER,ILOSTAT,DP Mean Hours Worked,DP Median Hours Worked
i64,i64,i64,f64,i64
2006,3,1,37.361034,29
2007,2,1,37.54387,48
2004,3,1,37.338537,34
2008,4,1,37.418794,48
2012,4,1,37.863292,39
…,…,…,…,…
2013,4,1,37.029466,39
2009,4,1,37.74394,54
2010,3,1,37.866828,40


## Comparison of Sequential and Parallel

Overall, parallel composition provides a much better bound than sequential composition. The total privacy loss for parallel composition is just ε since there are $k$ disjoint partitions and each of them also satisfies ε-dp. With sequential composition, the total privacy loss is $k$ * ε since we would run each query $k$ times.

However, one disadvantage of parallel composition is that has the dataset is split into more parts, each part will have a weaker signal and hence less accuracy. 

Additionally, in cases where we want statistics that reference the whole dataset, sequential composition may be a better fit. 

Therefore, the recommendation is to use parallel composition unless the partitions are too small to provide adequate accuracy or the entire data needs to be referenced for the calculated statistics. 

## Conclusion

This notebook illustrated: 

- The difference between interactive and noninteractive queries. 
- The definition and properties of sequential composition. 
- How to implement non-interactive sequential composition with polars, interactive sequential composition using transformations and measurements directly, and interactive sequential composition using polars queries. 
- The definition and properties of parallel composition. 
- How to implement noninteractive parallel composition and interactive parallel composition. 
- The differences between parallel and sequential composition and recommendations for when to choose which. 
